In [1]:
from __future__ import absolute_import, division, print_function
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import pickle
import csv
import random
import tensorflow as tf
from tensorflow.keras import Sequential
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM, Embedding, Reshape
print("Version: ", tf.__version__)

/home/nagato/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nagato/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nagato/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nagato/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: 

Version:  2.0.0-beta1


In [2]:
url="https://raw.githubusercontent.com/tlkh/text-emotion-classification/master/data.csv"
import urllib.request
urllib.request.urlretrieve(url, "data.csv")
sentence=[]
sentiment=[]
texts=[]
with open("data.csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        texts.append(row)
random.shuffle(texts)
print(texts[12])
for row in texts:
        sentence.append(row[0])
        sentiment.append(int(row[1]))
# neutral, happy, sad, anger, hate.

['its almost your birthday', '1']


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(sentence)
from tensorflow.keras.preprocessing.sequence import pad_sequences
sentence_token = tokenizer.texts_to_sequences(sentence)
train_x = pad_sequences(sentence_token[:44000], maxlen=20)
train_y= np.array(sentiment[:44000])
test_x = pad_sequences(sentence_token[44000:], maxlen=20)
test_y=np.array(sentiment[44000:])
print(train_x.shape)

(44000, 20)


In [4]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D
tf.keras.backend.clear_session()
model = Sequential()
model.add(Embedding(20000, 128))
model.add(tf.keras.layers.Conv1D(
                 100,
                 5,
                 padding='valid',
                 activation='relu',
                 strides=1))
# # we use max pooling:
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(50))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
optim=tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07,)
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=optim,
              metrics=['accuracy'])
print(model.summary())
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(train_x, train_y , batch_size=32, epochs=4, validation_data=(test_x, test_y), callbacks=[callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         2560000   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 100)         64100     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 100)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 50)                30200     
_________________________________________________________________
dense (Dense)                (None, 64)                3264      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 3

In [23]:
output = ["neutral", "happy", "sad", "anger", "hate"]
text="i hate working"
text_p = tokenizer.texts_to_sequences([text])
test = pad_sequences(text_p, maxlen=20)
print(text)
print(text,output[np.argmax(model.predict(pad_sequences(test)))])

i hate working
i hate working anger
